In [64]:
import numpy as np
from qiskit.quantum_info import Statevector
from qiskit.quantum_info import random_statevector
# from functools import reduce

# To Do

1. Update Criteria to normalize Psi
   1. 0 ket should have amplitude '1'
   2. or, divide all amplitudes by 0 ket amplitude
2. Finish decimal indices version
3. ~~Copy main functions to new notebook~~
4. Basis Change? Choose any ket from which to perform bit flip/basis change
   1. This is an equivalence relation

# Entanglement Criteria

## Full Algorithm

Input: a Statevector $|\psi\rangle$
- https://qiskit.org/documentation/stubs/qiskit.quantum_info.Statevector.html
- See Initialization of Statevectors (below)
- Algorithm only applies to Statevectors of size $2^n$ from a vector space $V^{\otimes n}$
- Note: Statevector will store amplitude of 0 for any kets whose amplitudes are not provided
  
1. Get n = number of qubits per ket
2. Get list of all basis-kets of length n
3. Get list of all non-basis kets of length n
4. For each non-basis ket:
   1. Get list of corresponding basis-kets
   2. Apply Criteria
   3. Record Results
5. If False occurs in Results, print "Entangled" message, Else Print "Not Entangled" message

### Determine All Basis Kets

Input: n = number of qubits per ket

1. Generate a list of powers of two from $2^0$ to $2^{n-1}$
2. Generate a list of kets `list(Psi.keys())` from Statevector dictionary keys
3. Use powers of two to reference basis kets in list of kets and create a new
   list of basis kets

### Determine All Non-Basis Kets

Input: list of basis kets, list of dictionary keys

1. Copy the list of dictionary keys, excluding all basis kets

### Determine Basis Kets corresponding to each Non-Basis Ket

Input: list of non-basis kets

For each ket:

1. For each bit of ketstring, check if bit is equal to '1'
2. If True, record index of bit
3. The corresponding basis ket is stored with same index in basis kets list
4. Generate a list of corresponding basis kets from all such indices
5. Store list in a dictionary indexed by key `'basis kets'`

### Create Dictionary of all Non-Basis Kets

1. Create dictionary with non-basis kets as keys
2. Values are the corresponding basis ket dictionary for each ket

### Criteria

Input: a non-basis ket index (decimal k or binary b)

1. If not provided, determine corresponding basis kets
2. Check if product of basis ket amplitudes = non-basis ket amplitude
3. Return dictionary with target amplitude and boolean result
4. If this is running for entire statevector, append these values to 
   non-basis ket dictionary

**Below only applies to `check_single_ket()` function when basis_kets are not provided**

#### Method 1: Binary Basis Kets (for Qiskit Statevector Dictionary)
Given a non-basis binary ketstring $b$ in $\{0,1\}^n$
- for each index $i$, check if `b[i] == "1"`
- when true:
  - initialize a new string of length $n$
  - use bitwise `|` to flip the bit at index $n-1-i$ (bitwise counts from R to L)
  - store result in new list
- the new list contains the $e_i$ basis ket strings corresponding to $b$
  
#### Method 2: Decimal Indices (for Qiskit Statevector)
Given a non-basis binary ketstring $b$ in $\{0,1\}^n$
- for each index $i$, check if `b[i] == "1"`
- when true:
  - append $i$ to a new list of indices
  - subtract each $i$ from $n-1$
- this new list is the place value exponents of 2 represented by each basis string
- create a new list of powers of 2 from the list of exponents
- The list of powers of 2 are the decimal indices corresponding to basis kets
- `int(b, 2)` gives the decimal index for $b$ in the Statevector

#### Check Product of Basis Ket Amplitudes

Input: list of basis ket indices

##### Using binary indices
1. Initialize product = 1
2. For all i in index_list:
   1. product = product*Statevector.to_dict[index]
   2. check whether product == Statevector.to_dict[b]

##### Using decimal indices
1. Initialize product = 1
2. For all i in index_list:
   1. product = product*Statevector[2^(n-1-i)]
3. check whether product == Statevector[k]

#### Notes
1. The values $2^{(n-1-i)}$ are indices that reference the amplitudes of the basis kets in the Qiskit `Statevector`.
   and reference binary keys in `list(Statevector.to_dict().keys())`.
2. The binary keys reference the basis ket amplitudes in `Statevector.to_dict()`.
3. For Statevectors of $n$ qubits of arbitrary length, a Qiskit Statevector can have many different subsystems
   - We can still embed the Statevector in a span of $2^n$ elements
   - Need to discuss what Entanglement means in this case



In [115]:
# Get list of Powers of Two in Binary or Decimal up to 2**(n-1)
# input: n = number of qubits, base (optional)
# output: list of powers of two up to 2**(number_qubits-1)

def powers_of_two(n, base=None):
    if base == 10:
        return [2**(n-1-i) for i in range(n)]
    else:
        return [format(1 << (n - 1 - i) | 0, '0'+str(n)+'b') for i in range(n)]


# Get list of Non-Basis Kets in Binary or Decimal
# input: n = number of qubits, list of powers of 2, base (optional)
# output: list of non-basis kets

def non_basis_kets(n, powers, base=None):
    if base == 10:
        non_basis_kets = [i for i in range(1,2**n) if i not in powers]
    else:
        non_basis_kets = [ket for ket in list(Psi.keys())[1:] if ket not in powers]
        
    return non_basis_kets


# test
number_qubits = 4
base = None

powers = powers_of_two(number_qubits, base)
print(powers)

non_basis_kets_list = non_basis_kets(number_qubits, powers, base)
print(non_basis_kets_list)

['1000', '0100', '0010', '0001']
['0011', '0101', '0110', '0111', '1001', '1010', '1011', '1100', '1101', '1110', '1111']


In [126]:
# Determine Basis Kets (binary)
# input:    
#   ket = non-basis ket
#   powers = list of all basis kets
# output:
#   list of basis kets

def get_basis_kets(ket, powers):
    basis_kets = [powers[index] for index, bit in enumerate(ket) if bit == "1"]
    return basis_kets

# Determine Basis Kets (decimal)
# creates a list of basis kets indices corresponding to non-basis ket
# input:
#   powers = list of powers of two
#   k = non-basis ket index
# output:
#   list of basis ket indices

def get_basis_indices(k, powers):
    k_list = []
    for p in powers:
        if k < p:
            pass
        else:
            k = k - p
            k_list.append(p)
    return k_list    

# test


In [117]:
# Create a dictionary indexed by all non-basis kets in Psi
#   initialize values as dictionaries of lists of corresponding of basis kets
# input:
#   list of non-basis kets (binary) or non-basis ket indices (decimal)
#   list of all basis kets (binary) or ket indices (decimal)
# output:
#   dictionary

def non_basis_kets_dict(list, powers, base=None):
    # list = list of non-basis kets or indices
    # powers = list of basis kets or powers of 2
    # base = 10 for indices output
    dict = {}
    if base == 10:
        # decimal function
        get_kets = get_basis_indices
    else:
        # binary function
        get_kets = get_basis_kets

    for ket in list:
        m = ket
        basis_kets = get_kets(ket, powers)
        dict[m] = { 'basis_kets': basis_kets }

    return dict

# test

# binary
dict = non_basis_kets_dict(non_basis_kets_list, powers)
print(dict)

{'0011': {'basis_kets': ['0010', '0001']}, '0101': {'basis_kets': ['0100', '0001']}, '0110': {'basis_kets': ['0100', '0010']}, '0111': {'basis_kets': ['0100', '0010', '0001']}, '1001': {'basis_kets': ['1000', '0001']}, '1010': {'basis_kets': ['1000', '0010']}, '1011': {'basis_kets': ['1000', '0010', '0001']}, '1100': {'basis_kets': ['1000', '0100']}, '1101': {'basis_kets': ['1000', '0100', '0001']}, '1110': {'basis_kets': ['1000', '0100', '0010']}, '1111': {'basis_kets': ['1000', '0100', '0010', '0001']}}


In [118]:
# Multiply Basis Ket Amplitudes
# input: statevector, list of basis kets (binary or decimal)
# output: product of amplitudes indexed by basis kets in Statevector dictionary
#           or basis ket indices in Statevector

def basis_amplitude_product(statevector, index_list):
    """ Works with decimal and binary indices """
    product = 1    
    for index in index_list:
        product = product*statevector[index]
    return product


# Equality check
# input: two values
# output: True or False
# Make lambda function?

def is_equal(a, b):
    return a == b

In [121]:
# single ket check function (using binary keys)
# input: statevector, ket, list of basis kets (optional)
#   if basis kets not provided, it will generate them
# output: dictionary with amplitude and boolean check value

def check_single_ket(statevector, ket, basis_kets=None):
    dict = {}
        
    if basis_kets == None:
        basis_kets = generate_basis_kets(ket)
        dict['basis_kets'] = basis_kets

    # get product of basis ket amplitudes and add to dictionary
    dict['target_amplitude'] = basis_amplitude_product(statevector, basis_kets)

    # check if ket amplitude = product of basis ket amplitudes and add to dictionary
    dict['equality'] = is_equal(statevector[ket], dict['target_amplitude'])

    # Print Statements
    def print_statements():
        product_string = basis_product_string(basis_kets)
        print_entanglement_equation(ket, product_string, dict['equality'])
        if dict['equality'] == False:
            print_basis_amplitudes(product_string, dict['target_amplitude'])
    
    # print_statements()
    
    return dict


# Generate basis kets
# Method 1 (binary)
# Use with check_single_ket and Statevector Dictionary when basis_kets not provided
# input: (binary) non-basis ket string
# output: (binary) keys for basis kets in Statevector Dictionary
def generate_basis_kets(ketstring):
    length = len(ketstring)
    basis_kets = [format(1 << (length - 1 - index) | 0, '0'+str(length)+'b')
               for index, bit in enumerate(ketstring) if bit == "1"]    
    return basis_kets

# Generate basis indices
# Method 2 (decimal)
# Use with check_single_ket and Statevector when basis_kets not provided
# input: (binary) non-basis ket string
# ouput: (decima) indices for basis kets in Statevector
def generate_basis_indices(ket):
    indices = [2**(len(ket) - 1 - index) for index, bit in enumerate(ket) 
    if bit == "1"]
    return indicies


# Print Statements for check_single_ket()

# Print Product of Basis Kets Expression
# input: list of basis kets for a non-basis ket
# output: string of product of Psi[basis_ket]
def basis_product_string(list):
    product = "Psi['"+list[0]+"']"
    for index in list[1:]:
        product = product + "*Psi['"+index+"']"
    return product

# Print Non-basis ket amplitude
# input: non-basis ket, amplitude
def print_ket_amplitude(ket, amplitude):
    print("Psi['"+ket+"'] = " + str(amplitude))

# Print Product of Basis Kets Amplitude
# input: product of basis ket string, amplitude
def print_basis_amplitudes(kets, amplitude):
    print(kets + " = " + str(amplitude))

# Print True/False check statement
# input: non-basis ket, product of basis ket string, boolean check value
def print_entanglement_equation(ket, basis_elements, bool):
    print("Psi['"+ket+"']" + " == " + basis_elements + " is " + str(bool))

# test generate basis ket with 4-qubit ket
ketstring = '1011'

# binary basis kets
basis_kets = create_basis_kets(ketstring)
print("Given " + str(ketstring) + ", Method 1 returns the list " + str(basis_kets))

# decimal basis indices
indices = ket_to_powers(ketstring)
print("Given " + str(ketstring) + ", Method 2 returns the list " + str(indices))

# test
single_ket = check_single_ket(Psi, '1011')
single_ket

Given 1011, Method 1 returns the list ['1000', '0010', '0001']
Given 1011, Method 2 returns the list [8, 2, 1]


{'basis_kets': ['1000', '0010', '0001'],
 'target_amplitude': (1.815207830942379-3.376890393151645j),
 'equality': False}

In [122]:
# Entanglement Function (binary kets)
# input: Statevector Dictionary
# output: dictionary 
#   keys: all non-basis kets
#   values: dictionaries of corresponding basis kets, target amplitude, boolean check

def entangled(statevector):
    # get number of qubits
    number_qubits = len(list(statevector.keys())[0])

    # generate list of basis kets:
    powers = powers_of_two(number_qubits)

    # generate list of non-basis kets:        
    non_basis_kets_list = non_basis_kets(number_qubits, powers)

    # store all non-basis kets in dictionary:
    dict = non_basis_kets_dict(non_basis_kets_list, powers)

    # initalize booleans
    booleans = set()

    for ket in dict:
        # get results fron check single ket:
        ket_results = check_single_ket(
            statevector, 
            ket, 
            dict[ket]['basis_kets']
            )
        # update dictionary with results:
        dict[ket]['target_amplitude'] = ket_results['target_amplitude']
        dict[ket]['equality'] = ket_results['equality']
        # faster way to do this?  items()?
        booleans.add(dict[ket]['equality'])

    # conclusion
    if False in booleans:
        print("|Psi> is Entangled")
    else:
        print("|Psi> is not Entangled")

    return dict

results = entangled(Psi)
results

|Psi> is Entangled


{'0011': {'basis_kets': ['0010', '0001'],
  'target_amplitude': (-1.8131728771753264+1.2768337324229941j),
  'equality': False},
 '0101': {'basis_kets': ['0100', '0001'],
  'target_amplitude': (1.4373792186376444+1.0694989959388226j),
  'equality': False},
 '0110': {'basis_kets': ['0100', '0010'],
  'target_amplitude': (2.388703565195332-2.0698270223130666j),
  'equality': False},
 '0111': {'basis_kets': ['0100', '0010', '0001'],
  'target_amplitude': (-1.1948854464441099-3.336198111268769j),
  'equality': False},
 '1001': {'basis_kets': ['1000', '0001'],
  'target_amplitude': (0.1818491638320242+1.929745058047091j),
  'equality': False},
 '1010': {'basis_kets': ['1000', '0010'],
  'target_amplitude': (3.3935787611518484+0.42004469055259785j),
  'equality': False},
 '1011': {'basis_kets': ['1000', '0010', '0001'],
  'target_amplitude': (1.815207830942379-3.376890393151645j),
  'equality': False},
 '1100': {'basis_kets': ['1000', '0100'],
  'target_amplitude': (-0.5337185484647919-2.710

In [ ]:
# TO DO
# Entanglement Function (decimal indices)
# input: Statevector
# output: dictionary 
#   keys: all non-basis ket indices
#   values: dictionaries of corresponding basis kets indices, target amplitude, 
#           boolean check

def entangled(statevector, ket=''):
    pass

def check_single_ket(statevector, ket):      
    """ Number of qubits per ket """
    number_qubits = len(ket)
    print("number of qubits = " + str(number_qubits))
    

    ### Using decimal indices
    """ Convert binary target ket string to statevector decimal index """
    decimal_index = int(ket, 2)
    print("target index = " + str(decimal_index))
    
    """ Get Statevector decimal indices for corresponding basis elements """
    basis_indices = ket_to_powers(ket)
    print("basis indices = " + str(basis_indices))

    """ Get target amplitude from Statevector """
    target_amplitude = statevector[decimal_index]
    print("target_amplitude = " + str(target_amplitude))
    
    """ Get product of basis ket amplitudes """
    product_basis_amplitudes = basis_amplitude_product(statevector, basis_indices)
    
number_qubits = 4
dims = 2**number_qubits
random_state = random_statevector(dims, None)
print(random_state)

check_single_ket(random_state, '1111')

Statevector([-0.18808617-0.1589944j ,  0.17075775-0.17173285j,
              0.10138651+0.22976637j, -0.01480501+0.09208322j,
             -0.07926826-0.28895903j, -0.08794139-0.15355775j,
             -0.16583805-0.17470529j, -0.14651142-0.11028345j,
              0.05402964+0.35868494j,  0.31217184+0.0476259j ,
              0.0824692 -0.21815926j, -0.01079228-0.23770372j,
              0.05027123-0.00191067j,  0.034295  +0.42969689j,
              0.06181807-0.0499862j ,  0.23662971+0.05664917j],
            dims=(2, 2, 2, 2))
number of qubits = 4
target index = 15
basis indices = [8, 4, 2, 1]
target_amplitude = (0.23662970843003236+0.056649168796742j)


## Initialize Statevector

### Option 1
- For an n-qubit state, initialize an array of amplitudes of size 
  $2^n$
- Convert the array to Qiskit Statevector `psi`
- Convert the array to Qiskit Statevector dictionary `Psi`

### Option 2
- For an n-qubit state, initialize a nonzero array of size $2^n$, e.g. `np.ones(2**n)`
- Convert the array to a Qiskit Statevector `psi`
- Convert Statevector to Qiskit Statevector dictionary `Psi`
- Populate the values of the Statevector dictionary with desired amplitudes

### Option 3
- Generate Qiskit random statevector
  
### Notes
The decimal indices of a Qiskit `Statevector` correspond to binary string 
keys in a Qiskit Statevector.to_dict() dictionary, e.g. 
`Statevector[5]` is the same as `Statevector.to_dict()['101']`.

In [78]:
# Option 1

# Example: Initialize a list of amplitudes equal to the list index
number_qubits = 4
amplitudes = [i for i in range(2**number_qubits)]    

# change zero ket amplitude to 1 to maintain our normalized zero ket condition
amplitudes[0] = 1

# create Statevector
psi = Statevector(amplitudes)
print(psi)

# Convert Statevector to dictionary
Psi = psi.to_dict()
print(Psi)


Statevector([ 1.+0.j,  1.+0.j,  2.+0.j,  3.+0.j,  4.+0.j,  5.+0.j,  6.+0.j,
              7.+0.j,  8.+0.j,  9.+0.j, 10.+0.j, 11.+0.j, 12.+0.j, 13.+0.j,
             14.+0.j, 15.+0.j],
            dims=(2, 2, 2, 2))
{'0000': (1+0j), '0001': (1+0j), '0010': (2+0j), '0011': (3+0j), '0100': (4+0j), '0101': (5+0j), '0110': (6+0j), '0111': (7+0j), '1000': (8+0j), '1001': (9+0j), '1010': (10+0j), '1011': (11+0j), '1100': (12+0j), '1101': (13+0j), '1110': (14+0j), '1111': (15+0j)}


In [79]:
# Option 2: Initialize Qiskit Statevector Dictionary and get amplitudes by user input

# Get amplitude of single ket
def getAmplitude(key, recursive=False):
    """ get ket amplitude by user input 
    checks if input is a number
    """
    if recursive:
        message = "Error: amplitude for " + str(key) + " must be a number: "
    else:
        message = "Enter an amplitude for " + str(key) + ": "
    
    amplitude = input(message)
    
    if amplitude == "":
        amplitude = 0

    try:
        amplitude = complex(amplitude)
    except ValueError:
        return getAmplitude(key, True)
    
    return amplitude

# Get amplitudes for all kets in Statevector
def getAmplitudes(n):
    # n = number of qubits
    """ Initialize nonzero Qiskit Statevector Dictionary of length 2^n """
    amplitudes = np.ones(2**n)
    statevector=Statevector(amplitudes).to_dict()
    """ Get amplitudes for each ket by user input """
    for key in statevector.keys():
        amplitude = getAmplitude(key)
        statevector[key] = amplitude    
    return statevector


#test

output = getAmplitude(1011)
print(output)
type(output)

statevector = getAmplitudes()
statevector

(1+0j)


{'0000': (1+0j),
 '0001': (2+0j),
 '0010': (3+0j),
 '0011': (4+0j),
 '0100': (5+0j),
 '0101': (6+0j),
 '0110': (7+0j),
 '0111': (8+0j),
 '1000': 0j,
 '1001': (2+0j),
 '1010': (452345+0j),
 '1011': (2+0j),
 '1100': (34+0j),
 '1101': 0j,
 '1110': 0j,
 '1111': 0j}

In [81]:
# Option 3

# Example: generate random state vector:
# https://entangledquery.com/t/how-to-generate-a-random-state-in-qiskit/21/

# Qiskit Quantum Information library:
# https://qiskit.org/documentation/apidoc/quantum_info.html

number_qubits = 4
dims = 2**number_qubits
      
random_state = random_statevector(dims, None)
print(random_state)

# Normalize zero ket to have amplitude = 1
normalized_random_state = random_state/random_state[0]
print(normalized_random_state)

# Convert normalized Statevector to dictionary:
Psi = normalized_random_state.to_dict()
print(Psi)

Statevector([-0.13661694+0.08832346j,  0.03692219+0.17861878j,
              0.32174689+0.00426943j,  0.15873036-0.12358027j,
             -0.0778907 -0.24801729j, -0.05862798-0.04422403j,
             -0.18884515-0.23859819j, -0.28569308+0.1329396j ,
              0.14286887-0.24225273j, -0.25718291-0.23695628j,
             -0.00216963-0.0791075j , -0.07830399+0.19622163j,
             -0.02750064+0.31545799j,  0.06445611+0.11233476j,
             -0.04618961-0.39892949j,  0.00388507+0.02791338j],
            dims=(2, 2, 2, 2))
Statevector([ 1.        -2.27460515e-17j,  0.40551451-1.04527546e+00j,
             -1.64665106-1.09581830e+00j, -1.23181638+1.08200282e-01j,
             -0.4256362 +1.54024555e+00j,  0.15505465+4.23951768e-01j,
              0.17855995+1.86191572e+00j,  1.91844984+2.67203468e-01j,
             -1.54598767+7.73738237e-01j,  0.53680808+2.08150632e+00j,
             -0.25280877+4.15604046e-01j,  1.05907384-7.51594680e-01j,
              1.19475264-1.53665644e+0

In [124]:
# Entanglement Criteria for 2 qubits
# Psi["01"]*Psi["10"] == Psi["11"]*Psi["00"]

In [125]:
# Convert list of decimals into binary strings of given length
# input: list of decimals, desired length of binary string
# output: binary strings padded with leading zeros to match desired length

def build_binary_string(list, length):
    """ Convert decimal ints to binary strings and pad strings with leading 
    zeros.
    
    Parameters
    ----------
    list : int
        Decimal values.
    length: int
        Number of qubits.

    Returns
    -------
    list : str
       Binary strings with padded zeros to reach specified length.
    """
    kets = [format(entry, '0'+str(length)+'b') for entry in list]
        
    return kets

# Generate list of binary strings of length n from list of decimal indices
number_qubits = 8
kets_list = build_binary_string(indices, number_qubits)
print(kets_list)



['00001000', '00000010', '00000001']


In [92]:
# Binary arithmetic
0b10**4
bin(0b10**4)

# Convert decimal integer to binary and remove Python's two leading bits
# Use Python format(number, '0nb') where n = length of desired bitstring

"""
def binary(number):
    return bin(number)[2:]

bitstring = binary(7)
print(bitstring)
"""

# Python's format() method

n = str(4)
binary = format(7, '0'+n+'b')
print(binary)

format(1 << 2 | 0, '0'+'8'+'b')

# type(bin(0b1011))
# type(0b1011)

0111


'00000100'